### 二分搜索
&emsp;&emsp;二分查找法是一种常见的查找方法，将原本线性时间提升到了对数时间范围，缩短了搜索时间，注意，二分搜索的前提是给定的序列是有序的，其实思路很简单，但是细节很复杂。现在就总结一下几个常用的二分查找场景：寻找一个数，寻找左侧边界，寻找右侧边界。而且深入细节研究，比如不等号是否应该带等号，mid是否应该加1等等，分析细节的差异以及出现这些差异的原因，对二分查找的理解更加深入。

#### 1.二分查找框架
```
def binarySearch(nums,target):
    left = 0
    right = ...
    while (...):
        mid = left + (right - left ) //2
        if nums[mid] == target:
            ...
        elif nums[mid] <  target:
            left = ...
        elif nums[mid] > target:
            right = ...
    return ...
```

&emsp;&emsp;分析二分查找的一个技巧就是将所有的情况都列举出来，可以清楚的显示所有细节。其中...标记的部分，就是可能出现细节问题的地方
#### 2.寻找一个数（基本的二分搜索）
&emsp;&emsp;搜索一个数，如果存在，返回索引，否则返回-1

In [7]:
def binarySearch(num_list,target):
    left,right = 0,len(num_list) - 1
    while left <= right:
        mid = left + (right - left) //2
        if num_list[mid] == target:
            return mid
        elif num_list[mid] > target:
            right = mid - 1
        else:
            left = mid + 1
    return -1

In [8]:
nums = [2,4,5,6,9]
target = 6
res_index = find(nums,target)

In [9]:
res_index

3

&emsp;&emsp;**问题**

&emsp;&emsp;1.为什么while循环的条件中是<= ,而不是<?

&emsp;&emsp;答：因为初始化right的赋值是len(nums) - 1,即最后一个元素的索引，而不是len(nums)，<= 和 < 可能出现在不同功能的二分查找中，区别是：前者相当于两端都是闭区间[left,right]，后者是左闭右开区间[left,right)，因为索引为len(nums)是越界的。在这个算法中使用的是前者[left,right]两端都闭的区间，这个区间就是每次进行搜索的区间。

&emsp;&emsp;什么时候应该停止搜索？找到目标值的时候可以终止：
```
if nums[mid] == target:
    return mid
```
&emsp;&emsp;但是如果没有找到，就需要while循转终止，然后返回-1,那么while循环应该什么时候终止呢？搜索区间为空的时候应该终止。while(left <= right)的终止条件是left == right + 1,写成区间的形式就是[right + 1,right]，这个时候区间为空，所以这时候终止while循环是正确的，直接返回-1.

&emsp;&emsp;while (left < right)的终止条件是left == right,写成区间的形式是[left,right]，此时区间还有一个数字未检索，但是此时while循环就终止了，也就是说此时区间中的数字还未检查，如果这个时候直接返回-1是错误的

&emsp;&emsp;如果要使用while(left < right),打个补丁如下:
```
while left < right:
    ...
return nums[left] == target ?left :-1
```

&emsp;&emsp;2.为什么是left = mid + 1,right = mid - 1,有的代码是left = mid ，right = mid ,这是怎么回事，如何判断?

&emsp;&emsp;答：刚刚明确了搜索区间的概念，而且本算法的搜索区间的两端都是闭合的，即[left,right]，那么当发现索引mid不是要找的target，如何确定下一步的搜索空间？当然是[left ,mid - 1] or [mid + 1,right]，因为mid已经搜索过，应该从搜索空间去除。但是此算法含有一定的缺陷，如果给定有序数组nums = [1,2,2,2,3],target = 2 ,此算法返回的索引是2，没错，但是如果向得到target的左侧边界，即索引1，或者想得到target的右侧边界，即索引3，这种需求目前的算法是无法处理的。

#### 2.寻找左侧边界的二分搜索
&emsp;&emsp;代码如下

In [10]:
def left_bound(nums,target):
    if len(nums) == 0:return -1
    left,right = 0,len(nums)
    while left < right:
        mid = left + (right - left) //2
        if nums[mid] == target:
            right = mid
        elif nums[mid] < target:
            left = mid + 1
        elif nums[mid] > target:
            right = mid
            
    return left

In [11]:
nums = [1,2,2,2,3]
target = 2
res_index = left_bound(nums,target)
print(res_index)

1


&emsp;&emsp;1.为什么while循环的条件中是< ,而不是<=\?

&emsp;&emsp;答：因为一开始right == len(nums)，而不是len(nums)，因此每次搜索的区间是左闭右开区间[left,right)，while（left < right）的终止条件是left == right，此时搜索区间[left,right)为空，所以可以正确终止。

&emsp;&emsp;2.为什么没有返回-1的操作，如果nums中不存在target这个值，怎么办？

&emsp;&emsp;答：对于左侧边界的特殊含义，对于nums = [1,2,2,2,3],target = 2,对于这个数组，算法会返回1，这个1的含义可以理解为nums中小于2的元素有1个。比如对于有序数组nums = [2,3,5,7],target = 1，算法返回0，含义是nums中小于1的元素有0个，在比如nums不变，target= 8，此时算法会返回4，含义是nums中小于8的元素有4个。综上可以看出，函数的返回值（left变量的值）取值区间是闭区间[0,len(nums)],所以简单添加两行代码就能在正确的时候return -1。
```
while left < right:
    ...
if left == len(nums):return -1
return nums[left] == target?left:-1 
```

&emsp;&emsp;3.为什么是left = mid + 1,right = mid?和之前的算法不一样？

&emsp;&emsp;答：因为此时的搜索区间是左闭右开的，所以当nums[mid]被检测之后，下一步的搜索区间应该是去掉mid分割成两个区间，即[left,mid)或[mid+1,right)

&emsp;&emsp;4.为什么该算法能够搜索左侧边界？

&emsp;&emsp;答：关键对于nums[mid] == target这种情况的处理：
```
if nums[mid] == target:
    right = mid
```
&emsp;&emsp;可以看出，在找到target的时候不要立即返回，而是缩小搜索区间的上界right，在区间[left,mid)中继续搜索，即不断向左搜索，达到锁定左侧边界的目的。

&emsp;&emsp;5.为什么返回left而不是right？

&emsp;&emsp;答：跳出while循环的条件是left == right,所以返回left和返回right是一致的。

#### 3.寻找右侧边界的二分搜索
&emsp;&emsp;代码如下

In [12]:
def right_bound(nums,target):
    if len(nums) == 0:return -1
    left = 0
    right = len(nums)
    while left < right:
        mid = left + (right - left) // 2
        if nums[mid] == target:
            left = mid + 1
        elif nums[mid] < target:
            left = mid + 1
        elif nums[mid] > target:
            right = mid
    return left - 1

In [13]:
nums = [1,2,2,2,3]
target = 2
res_index = right_bound(nums,target)
print(res_index)

3


&emsp;&emsp;1.为什么这个算法能够找到右侧边界?

&emsp;&emsp;答：关键代码如下:
```
if nums[mid] == target:
    left = mid + 1
```
&emsp;&emsp;当nums[mid] == target时，不要立即返回，而是增大搜索区间的下界left，使得区间不断向右收缩，达到锁定右侧边界的目的。

&emsp;&emsp;2.为什么最后返回left - 1,而不像左侧边界的函数，返回left?

&emsp;&emsp;答：为什么要减一，这是搜索右侧边界的一个特殊点，关键在于这个条件的判断：
```
if nums[mid] == target:
    left = mid + 1
```
&emsp;&emsp;因为对left的更新是left = mid + 1，就是说while循环结束的时候，nums[left] 一定不等于target，而nums[left - 1] 可能是target。

&emsp;&emsp;3.为什么最后没有返回-1，如果nums中不存在target这个值怎么办？

&emsp;&emsp;答：添加两行代码
```
while left < right:
    ...
if left == 0:return -1
return nums[left-1] == target?left - 1:-1
```

#### 4.最后总结
&emsp;&emsp;梳理一下这些细节差异的因果逻辑：

1.最基本的二分查找算法：

    因为初始化right = len(nums) -1
    所以决定了搜索区间是[left,right]
    所以决定了循环条件是while(left <= right)
    同时也决定了left = mid + 1 和 right = mid - 1
    因为只需要找到一个target的索引即可
    所有当nums[mid] == target 的时候可以立即返回

2.寻找左侧边界的二分查找

    因为初始化right = len(nums)
    所以决定了搜索区间是[left,right)
    所以决定了循环条件是while (left<right)
    同时也决定了left = mid + 1和right = mid
    我们需要找到target的最左侧索引
    所以当nums[mid] == target 时不要立即返回
    而是要收紧右侧边界以锁定左侧边界

3.寻找右侧边界的二分查找

    因为初始化right = len(nums)
    所以决定了搜索区间是[left,right)
    所以决定了循环条件是while (left < right)
    同时也决定了left = mid + 1 和 right = mid
    我们需要找到target的最右侧索引
    所以当nums[mid] == target时不要立即返回
    而是要收紧左侧以锁定右侧边界

#### 5.二分搜索补充(个人感觉上面的方法不太明确，找到了一个更好的方案，进行补充)

In [1]:
#c++ 标准库中的bug_free的通用写法
#即使区间为空，答案不存在，有重复元素，搜索开/闭的上/下接也同样适用
def lower_bound(array,first,last,value):
    while first < last:
        mid = first = (last - first) //2
        if array[mid] < value:
            first = mid + 1
        else:
            last = mid
        return first

&emsp;&emsp;位置调整只出现了一次，而且最后返回first 或者 last都是正确的，因为搜索区间是左闭右开的。

&emsp;&emsp;二分搜索写法可以分为求上界和求下界情况两种，并转换为以下的等价写法。

&emsp;&emsp;1.求下界

&emsp;&emsp;即找出满足x >= value 或 x > value的最小的x的位置，用左闭右开搜索区间[first,last), 区间为空时终止并返回fisrt或last(first == last),求中点时从下界first闭区间侧出发，mid = first + (last - first)//2,以确保区间长度为1时，mid = first仍然在[first,last)区间内。

&emsp;&emsp;2.求上界

&emsp;&emsp;即找出满足x < = value 或 x < value 条件的最大x的范围，可以调用互补的求下界的函数在减一得到，如x >= value 的下界在减一 就是x < value的上界

&emsp;&emsp;**怎样理解这简单的6行代码**

    1.搜索范围[first,last)不为空，即first < last
    2.搜索范围[first,last)左侧，即[first0,first)内所有元素(若存在)，都小于value,其中first0是first的初始值
    3.搜索范围[first,last)右侧，即[last,last0)内所有元素(若存在)，都大于等于value,其中last0是last的初始值
    


&emsp;&emsp;分析代码if array[mid] < value: 说明mid应该是搜索区间的左侧区间，因此下一个搜索区间是first = mid + 1 其他情况是array[mid] >= value 说明mid应该是搜索区间的右侧，last = mid ，当长度为1时，依然在循环内，继续检查，最后一个元素一定是在左侧或者右侧，此时first == last。

#### 6.二分搜索 leetcode 例题

| 基础   |                                         |
|------|-----------------------------------------|
| 278  | First Bad Version                       |
| 35   | Search Insert Position                  |
| 33   | Search in Rotated Sorted Array          |
| 81   | Search in Rotated Sorted Array II       |
| 153  | Find Minimum in Rotated Sorted Array    |
| 154  | Find Minimum in Rotated Sorted Array II |
| 162	 | Find Peak Element                       |
| 374  | Guess Number Higher or Lower            |
| 34   | Search for a Range                      |
| 350  | Intersection of Two Arrays II           |
| 315  | Count of Smaller Numbers After Self     |
| 349  | Intersection of Two Arrays              |
| 300  | Longest Increasing Subsequence          |
| 354  | Russian Doll Envelopes                  |
